<a href="https://colab.research.google.com/github/rushikeshnaik779/PracticeForNLP/blob/main/ChatBot%20V1/ChatBot_Building_My_OwnChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import gensim 

print(gensim.__version__)

3.6.0


In [2]:
from tqdm import tqdm

class TqdmUpto(tqdm):

    def Update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None : self.total = tsize
        self.update(b * bsize - self.n)
        

In [4]:
def get_data(url, filename):
    """
    Download Data if the filename doesnot exist already 
    Uses Tqdm to show download progress 

    """
    import os 
    from urllib.request import urlretrieve

    if not os.path.exists(filename):
        dirname = os.path.dirname(filename)
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        with TqdmUpto(unit = 'B', unit_scale=True, miniters=1, desc=url.split('/')[-1]) as t:
            urlretrieve(url, filename, reporthook=t.Update_to)
        
    else : 
        print("Already Exist")
    
embedding_url = 'http://nlp.stanford.edu/data/glove.6B.zip'
get_data(embedding_url, 'data/glove.6B.zip')


glove.6B.zip: 862MB [02:42, 5.30MB/s]                           


In [7]:
 !unzip data/glove.6B.zip 
 !mv -v glove.6B.300d.txt data/glove.6B.300d.txt 
 !mv -v glove.6B.200d.txt data/glove.6B.200d.txt 
 !mv -v glove.6B.100d.txt data/glove.6B.100d.txt 
 !mv -v glove.6B.50d.txt data/glove.6B.50d.txt

Archive:  data/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
renamed 'glove.6B.300d.txt' -> 'data/glove.6B.300d.txt'
renamed 'glove.6B.200d.txt' -> 'data/glove.6B.200d.txt'
renamed 'glove.6B.100d.txt' -> 'data/glove.6B.100d.txt'
renamed 'glove.6B.50d.txt' -> 'data/glove.6B.50d.txt'


In [8]:
from gensim.scripts.glove2word2vec import glove2word2vec

glove_input_file = 'data/glove.6B.300d.txt'
word2vec_output_file = 'data/glove.6B.300d.txt.word2vec'

import os 
if not  os.path.exists(word2vec_output_file):
    glove2word2vec(glove_input_file, word2vec_output_file)

In [9]:
%%time 

from gensim.models import KeyedVectors
filename = word2vec_output_file
embed = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

CPU times: user 1min 58s, sys: 3.55 s, total: 2min 2s
Wall time: 2min


In [10]:
assert embed['awesome'] is not None


In [11]:
embed['for']

array([-2.4132e-01,  1.2063e-01,  1.9190e-01, -2.6692e-01,  6.1076e-02,
       -2.2878e-02,  5.8017e-01, -2.2533e-01, -2.0644e-01, -1.6361e+00,
        2.7715e-01,  1.5026e-01, -3.1691e-01,  3.8411e-01,  3.2003e-02,
       -1.2901e-01, -5.3065e-02, -5.5874e-03,  1.0467e-01, -4.2931e-01,
       -2.3605e-01,  2.6159e-01,  7.0261e-02,  4.6544e-02, -2.0760e-01,
       -2.3013e-01,  1.8501e-02,  2.5844e-01,  1.2839e-01,  1.6461e-01,
       -2.3486e-02,  2.0511e-01,  1.8169e-01,  3.0549e-01, -1.1893e+00,
       -4.0416e-01, -1.9842e-01,  1.4848e-01,  1.3756e-01, -9.3425e-02,
        1.9290e-01, -4.9650e-02,  3.8317e-02, -2.9397e-01, -7.9036e-02,
       -1.0161e-01,  1.1877e-01,  4.8220e-01, -2.9784e-01,  2.1025e-01,
        1.2835e-01, -2.6482e-01,  3.6112e-01, -3.0893e-01, -4.4381e-01,
        1.8737e-01, -2.1541e-01,  3.9490e-01, -2.2203e-02, -3.5489e-01,
       -3.6777e-01,  1.4181e-01,  4.1243e-01, -7.1339e-01, -7.3703e-02,
       -3.3782e-01,  2.0737e-01, -2.8501e-01,  4.7233e-01, -1.52

# BOT : FOOD ORDER

In [12]:
cuisine_refs = ["mexican", "thai", "british", "american", "italian"]
sample_sentence = "I am looking for a cheap Indian or Chinese place in Indiranagar" 

In [17]:
tokens = sample_sentence.split()
tokens = [x.lower().strip() for x in tokens]

threshold = 18.3
found = []
for term in tokens:
    if term in embed.vocab: 
        scores = []
        for C in cuisine_refs: 
            scores.append(np.dot(embed[C], embed[term].T)) 

        mean_score = np.mean(scores)
        print(f"{term} : {mean_score}")

        if mean_score > threshold: 
            found.append(term)
print(found) 

i : 8.132120132446289
am : 6.354741096496582
looking : 7.448504447937012
for : 10.627421379089355
a : 11.809560775756836
cheap : 7.09670877456665
indian : 18.64516258239746
or : 9.692893981933594
chinese : 19.09498405456543
place : 7.651237487792969
in : 10.085711479187012
['indian', 'chinese']


In [18]:
# LET"S CLASSIFY INTENT 
def sum_vecs(embed, text):
    tokens = text.split(' ')
    vec = np.zeros(embed.vector_size)

    for idx, term in enumerate(tokens):
        if term in embed.vocab : 
            vec = vec + embed[term]
    return vec


sentence_vector = sum_vecs(embed, sample_sentence)
print(sentence_vector.shape)

(300,)


In [19]:
data={
  "greet": {
    "examples" : ["hello","hey there","howdy","hello","hi","hey","hey ho"],
    "centroid" : None
  },
  "inform": {
    "examples" : [
        "i'd like something asian",
        "maybe korean",
        "what mexican options do i have",
        "what italian options do i have",
        "i want korean food",
        "i want german food",
        "i want vegetarian food",
        "i would like chinese food",
        "i would like indian food",
        "what japanese options do i have",
        "korean please",
        "what about indian",
        "i want some chicken",
        "maybe thai",
        "i'd like something vegetarian",
        "show me french restaurants",
        "show me a cool malaysian spot",
        "where can I get some spicy food"
    ],
    "centroid" : None
  },
  "deny": {
    "examples" : [
      "nah",
      "any other places ?",
      "anything else",
      "no thanks"
      "not that one",
      "i do not like that place",
      "something else please",
      "no please show other options"
    ],
    "centroid" : None
  },
    "affirm":{
        "examples":[
            "yeah",
            "that works",
            "good, thanks",
            "this works",
            "sounds good",
            "thanks, this is perfect",
            "just what I wanted"
        ],
        "centroid": None
    }

}

In [20]:
def get_centroid(embed, examples):
    C = np.zeros((len(examples), embed.vector_size))

    for idx, text in enumerate(examples):
        C[idx,:] = sum_vecs(embed, text)

    
    centroid = np.mean(C, axis=0)
    assert centroid.shape[0] == embed.vector_size
    return centroid 

In [21]:
for label in data.keys():
    data[label]["centroid"] = get_centroid(embed, data[label]["examples"])

In [23]:
for label in data.keys():
    print(f"{label}: {data[label]['examples']}")

greet: ['hello', 'hey there', 'howdy', 'hello', 'hi', 'hey', 'hey ho']
inform: ["i'd like something asian", 'maybe korean', 'what mexican options do i have', 'what italian options do i have', 'i want korean food', 'i want german food', 'i want vegetarian food', 'i would like chinese food', 'i would like indian food', 'what japanese options do i have', 'korean please', 'what about indian', 'i want some chicken', 'maybe thai', "i'd like something vegetarian", 'show me french restaurants', 'show me a cool malaysian spot', 'where can I get some spicy food']
deny: ['nah', 'any other places ?', 'anything else', 'no thanksnot that one', 'i do not like that place', 'something else please', 'no please show other options']
affirm: ['yeah', 'that works', 'good, thanks', 'this works', 'sounds good', 'thanks, this is perfect', 'just what I wanted']


In [24]:
def get_intent(embed, data, text):
    intents = list(data.keys())
    vec = sum_vecs(embed, text)
    scores = np.array([ np.linalg.norm(vec-data[label]["centroid"]) for label in intents ])
    return intents[np.argmin(scores)]

In [25]:
for text in ["hey ","i am looking for chinese food","not for me", "ok, this is good"]:
    print(f"text : '{text}', predicted_label : '{get_intent(embed, data, text)}'")

text : 'hey ', predicted_label : 'greet'
text : 'i am looking for chinese food', predicted_label : 'inform'
text : 'not for me', predicted_label : 'deny'
text : 'ok, this is good', predicted_label : 'affirm'


In [26]:
for text in ["Machine Learning University - Accelerated Natural Language Processing - Lectures go from introduction to NLP and text processing to Recurrent Neural Networks and Transformers. Material can be found here."]:
    print(f"text : '{text}', predicted_label : '{get_intent(embed, data, text)}'")

text : 'Machine Learning University - Accelerated Natural Language Processing - Lectures go from introduction to NLP and text processing to Recurrent Neural Networks and Transformers. Material can be found here.', predicted_label : 'inform'


# RESPONSES

In [27]:
templates = {
        "utter_greet": ["hey there!", "Hey! How you doin'? "],
        "utter_options": ["ok, let me check some more"],
        "utter_goodbye": ["Great, I'll go now. Bye bye", "bye bye", "Goodbye!"],
        "utter_default": ["Sorry, I didn't quite follow"],
        "utter_confirm": ["Got it", "Gotcha", "Your order is confirmed now"]
    }

In [28]:
response_map = {
    "greet": "utter_greet",
    "affirm": "utter_goodbye",
    "deny": "utter_options",
    "inform": "utter_confirm",
    "default": "utter_default",
}

In [29]:
import random 
def get_bot_response(bot_response_map, bot_templates, intent):
    if intent not in list(response_map):
        intent = "default"
    select_template = bot_response_map[intent]
    templates = bot_templates[select_template]
    return random.choice(templates)    

In [35]:
user_intent = get_intent(embed, data, "show me menu")
get_bot_response(response_map, templates, user_intent)

"Great, I'll go now. Bye bye"

In [37]:
for i in range(5):
    text = str(input())
    user_intent = get_intent(embed, data, text)
    bot_reply = get_bot_response(response_map, templates, user_intent)
    print(f"text : '{text}', intent: {user_intent}, bot: {bot_reply}")

hey
text : 'hey', intent: greet, bot: Hey! How you doin'? 
i am looking for south indian food
text : 'i am looking for south indian food', intent: inform, bot: Got it
not for me
text : 'not for me', intent: deny, bot: ok, let me check some more
ok, this is good
text : 'ok, this is good', intent: affirm, bot: bye bye
bye
text : 'bye', intent: greet, bot: hey there!
